In [1]:
#대표적 Unsupervised learning
#Generator->가짜 이미지를 만들어 내는 역할 (실제 데이터 분포와 유사한 데이터 분포를 만들어 내는 것이 역할)
#Discriminator-> 진짜일수록 1에 가까운 이미지
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
num_steps=10000
batch_size=128
learning_rate=0.0002
#G,D-DNN{256 hidden nodes}
image_dim=784
gen_hidden_dim=256
disc_hidden_dim=256
noise_dim=100

In [3]:
def he_init(shape):
    return tf.random_normal(shape=shape,stddev=1./tf.sqrt(shape[0]/2))
weights ={
    'gen_hidden1':tf.Variable(he_init([noise_dim,gen_hidden_dim])),
    'gen_out':tf.Variable(he_init([gen_hidden_dim,image_dim])),
    'disc_hidden1':tf.Variable(he_init([image_dim,disc_hidden_dim])),
    'disc_out':tf.Variable(he_init([disc_hidden_dim,1]))
}
biased= {
    'gen_hidden1':tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out':tf.Variable(tf.zeros([image_dim])),
    'disc_hidden1':tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc_out':tf.Variable(tf.zeros([1]))
}


In [4]:
def generator(x):
    hidden_layer = tf.matmul(x,weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer,biased['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.matmul(hidden_layer,weights['gen_out'])
    output_layer = tf.add(output_layer,biased['gen_out'])
    output_layer = tf.nn.sigmoid(output_layer)
    return output_layer

def discriminator(x):
    hidden_layer = tf.matmul(x,weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer,biased['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    output_layer = tf.matmul(hidden_layer,weights['disc_out'])
    output_layer = tf.add(output_layer,biased['disc_out'])
    output_layer = tf.nn.sigmoid(output_layer)
    return output_layer

In [5]:
gen_input=tf.placeholder(tf.float32,shape=[None,noise_dim],name='input_noise')
disc_input=tf.placeholder(tf.float32,shape=[None,image_dim],name='disc_input')

gen_sample=generator(gen_input)
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample)

gen_loss= -tf.reduce_mean(tf.log(disc_fake))

disc_loss = -tf.reduce_mean(tf.log(disc_real)+tf.log(1-disc_fake))#log(D(x))+log(1-D(G(x)))

optimizer_gen=tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

gen_var=[weights['gen_hidden1'],weights['gen_out'],biased['gen_hidden1'],biased['gen_out']]
disc_var=[weights['disc_hidden1'],weights['disc_out'],biased['disc_hidden1'],biased['disc_out']]
train_gen=optimizer_gen.minimize(gen_loss,var_list=gen_var)
train_disc=optimizer_disc.minimize(disc_loss,var_list=disc_var)

In [7]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(num_steps):
    batch_xs,_=mnist.train.next_batch(batch_size)
    z=np.random.uniform(-1.,1.,size=[batch_size,noise_dim])
    _,_,gl,dl=sess.run([train_gen,train_disc,gen_loss,disc_loss],feed_dict={disc_input:batch_xs,gen_input:z})
    if i%100==0:
        print('Step: ',i,'G Loss: ',gl,'D loss: ',dl)
    

Step:  0 G Loss:  0.51023096 D loss:  1.6910877
Step:  100 G Loss:  4.304199 D loss:  0.0562232
Step:  200 G Loss:  3.4847608 D loss:  0.16653407
Step:  300 G Loss:  2.6270256 D loss:  0.29272982
Step:  400 G Loss:  2.9421144 D loss:  0.18517986
Step:  500 G Loss:  3.1210232 D loss:  0.1304935
Step:  600 G Loss:  3.401279 D loss:  0.10034922
Step:  700 G Loss:  3.4512744 D loss:  0.07740161
Step:  800 G Loss:  3.5192213 D loss:  0.07086007
Step:  900 G Loss:  3.745284 D loss:  0.048982583
Step:  1000 G Loss:  3.7151065 D loss:  0.06478716
Step:  1100 G Loss:  3.804489 D loss:  0.057835408
Step:  1200 G Loss:  3.7682762 D loss:  0.10274626
Step:  1300 G Loss:  3.7630925 D loss:  0.08882432
Step:  1400 G Loss:  3.7492514 D loss:  0.06193714
Step:  1500 G Loss:  3.9610198 D loss:  0.08357944
Step:  1600 G Loss:  4.0925045 D loss:  0.06417206
Step:  1700 G Loss:  3.8507118 D loss:  0.055582985
Step:  1800 G Loss:  4.8300495 D loss:  0.06652072
Step:  1900 G Loss:  4.4171267 D loss:  0.0470